In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

In [2]:
def load_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    print("train shape:{}\ntest shape:{}".format(train.shape, test.shape))
    
    train_df = train.drop(['label'], axis=1)
    label = pd.get_dummies(train['label'])
    #train_df = train_df/255.0
    #test = test/255.0
    train_df = train_df.applymap(lambda x: x/255).astype(np.float32)
    test = test.applymap(lambda x: x/255).astype(np.float32)
    
    # Reshape image in 3 dimensions (height = 28px, width = 28px , canal = 1)
    train_df = train_df.values.reshape(-1,28,28,1)
    test = test.values.reshape(-1,28,28,1)
    
    x_train, x_valid, y_train, y_valid = train_test_split(train_df, label, test_size=0.2, random_state=2017)
    print("x_train shape:"+str(x_train.shape))
    print("x_valid shape:"+str(x_valid.shape))
    
    return x_train, y_train, x_valid, y_valid, test

In [3]:
x_train, y_train, x_valid, y_valid, x_test = load_data()

train shape:(42000, 785)
test shape:(28000, 784)
x_train shape:(33600, 28, 28, 1)
x_valid shape:(8400, 28, 28, 1)


In [20]:
# 牛逼的Sequential类可以让我们灵活地插入不同的神经网络层
model = Sequential()

# 加上一个2D卷积层， 32个输出（也就是卷积通道），激活函数选用relu，
# 卷积核的窗口选用3*3像素窗口
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(Conv2D(32, (3, 3), activation='relu'))

# 池化层是2*2像素的
model.add(MaxPooling2D(pool_size=(2, 2)))
# 对于池化层的输出，采用0.35概率的Dropout
model.add(Dropout(0.35))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
# 池化层是2*2像素的
model.add(MaxPooling2D(pool_size=(2, 2)))
# 对于池化层的输出，采用0.35概率的Dropout
model.add(Dropout(0.35))

# 展平所有像素，比如[28*28] -> [784]
model.add(Flatten())
model.add(Dense(512, activation='relu'))
# 对所有像素使用全连接层，输出为128，激活函数选用relu
model.add(Dense(256, activation='relu'))
# 对所有像素使用全连接层，输出为64，激活函数选用relu
model.add(Dense(64, activation='relu'))

# 对输入采用0.5概率的Dropout
model.add(Dropout(0.5))
# 对刚才Dropout的输出采用softmax激活函数，得到最后结果0-9
model.add(Dense(10, activation='softmax'))

# Define the optimizer
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50, batch_size=64, verbose=1, validation_data=(x_valid, y_valid))

# loss: 0.0293 - acc: 0.9915 - val_loss: 0.0323 - val_acc: 0.9925
# LB -> 99.14%

Train on 33600 samples, validate on 8400 samples
Epoch 1/50
33600/33600 [==============================] - 13s 396us/step - loss: 0.8678 - acc: 0.7172 - val_loss: 0.1705 - val_acc: 0.9485
Epoch 2/50
33600/33600 [==============================] - 12s 368us/step - loss: 0.2728 - acc: 0.9214 - val_loss: 0.1032 - val_acc: 0.9687
Epoch 3/50
33600/33600 [==============================] - 12s 360us/step - loss: 0.1954 - acc: 0.9455 - val_loss: 0.0822 - val_acc: 0.9745
Epoch 4/50
33600/33600 [==============================] - 13s 380us/step - loss: 0.1519 - acc: 0.9584 - val_loss: 0.0683 - val_acc: 0.9783
Epoch 5/50
33600/33600 [==============================] - 12s 369us/step - loss: 0.1265 - acc: 0.9654 - val_loss: 0.0590 - val_acc: 0.9825
Epoch 6/50
33600/33600 [==============================] - 13s 382us/step - loss: 0.1095 - acc: 0.9691 - val_loss: 0.0586 - val_acc: 0.9823
Epoch 7/50
33600/33600 [==============================] - 12s 367us/step - loss: 0.0934 - acc: 0.9745 - val_loss: 0.0

In [21]:
from datetime import datetime

# predict results
results = model.predict(x_test)

# select the indix with the maximum probability
results = np.argmax(results,axis = 1)

submit_df= pd.read_csv('../input/sample_submission.csv')
submit_df.Label = results
filename = "../sub/{}.scv".format(datetime.now().strftime('%Y%m%d_%H_%M'))
submit_df.to_csv(filename,index=None,encoding='utf-8')